In [2]:
# model = Equation (coeff, intercept)

In [ ]:
# save model
# portable ship
# load model
# predict

In [ ]:
train 
test

Hackathon
train+test -> Preprocessing (Hackathon)

train - flight price - carrier - indigo, airindia, jet

testflight price - carrier - indigo, airindia, jet, vistara


combined and onehot -> no problem in forseen

missing value -> imputation -> mean -> train+test


Realworld
Train - Preprocessing



Test - We don't have - Preprocess -> predict


Extra Burden on the devepoler to repeat the step of preprocessing 

Ship the processing code with the model



train - flight price - carrier - indigo, airindia, jet

test - flight price - carrier - indigo, airindia, jet, vistara


separatly did the onehot -> problem 


missing value -> imputation -> mean -> train



In [89]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler


from sklearn.linear_model import LinearRegression

import joblib

In [127]:
data = pd.DataFrame(
{
    'name':['Maruti','honda','maruti','toyOta','hondA','toyota'],
    'fuel_type':['petrol','diesel','petrol','diesel','petrol','diesel'],
    'km_driven' :[2000,4000,100,6344,1234,np.NaN],
    'price' :[200000,400000,424232,800000,500000,900000]
    
}
)
 

In [128]:
data

,name,fuel_type,km_driven,price
0,Maruti,petrol,2000.0,200000
1,honda,diesel,4000.0,400000
2,maruti,petrol,100.0,424232
3,toyOta,diesel,6344.0,800000
4,hondA,petrol,1234.0,500000
5,toyota,diesel,NaN,900000


In [129]:
data['name'].unique()

array(['Maruti', 'honda', 'maruti', 'toyOta', 'hondA', 'toyota'],
      dtype=object)

In [98]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [132]:
data['name'].apply(lambda x: x.lower())

0    maruti
1     honda
2    maruti
3    toyota
4     honda
5    toyota
Name: name, dtype: object

In [186]:
class DataframeFunctionTransformer():
    def __init__(self, func):
        self.func = func

    def transform(self, df, **transform_params):
        return self.func(df)

    def fit(self, X, y=None, **fit_params):
        return self
    

In [187]:
def preprocess_df(df):
    df['name'] = df['name'].apply(lambda x: x.lower())
    df['fuel_type'] = df['fuel_type'].apply(lambda x: x.lower())
    return df
    
    

In [188]:
initial_pipeline = Pipeline([
    ('pre_preprocess', DataframeFunctionTransformer(preprocess_df))
])

In [189]:
initial_pipeline.transform(data)

,name,fuel_type,km_driven,price
0,maruti,petrol,2000.0,200000
1,honda,diesel,4000.0,400000
2,maruti,petrol,100.0,424232
3,toyota,diesel,6344.0,800000
4,honda,petrol,1234.0,500000
5,toyota,diesel,NaN,900000


In [197]:
numerical_imput = Pipeline([('imputation', SimpleImputer(strategy='mean'))])
categorical_onehot = Pipeline([('onehot', OneHotEncoder(sparse = False,handle_unknown='ignore'))])
numerical_scaling = Pipeline([('scale', MinMaxScaler())])



preprocess_categorical_encoding = Pipeline([
                                       ('pre_preprocess', DataframeFunctionTransformer(preprocess_df)),
                                       ('onehot', categorical_onehot())])

numerical_encoding_scaling = Pipeline([
                                       ('imputation', SimpleImputer(strategy='mean')),
                                       ('scale', MinMaxScaler())])


TypeError: 'Pipeline' object is not callable

In [191]:
data.columns

Index(['name', 'fuel_type', 'km_driven', 'price'], dtype='object')

In [181]:
preprocess_full = ColumnTransformer(
    transformers=
    [
    ('numerical_prprocessing',numerical_encoding_scaling,['km_driven'],),
    ('categoral_preprocessing',preprocess_categorical_encoding,['name','fuel_type'])
    
    ]
)

In [182]:
pipe = Pipeline([
         ('preprocess',preprocess_full),
         ('model',LinearRegression())])

In [183]:
data.drop(columns = 'price')

,name,fuel_type,km_driven
0,maruti,petrol,2000.0
1,honda,diesel,4000.0
2,maruti,petrol,100.0
3,toyota,diesel,6344.0
4,honda,petrol,1234.0
5,toyota,diesel,NaN


In [184]:
pipe.fit(data.drop(columns = 'price'),y)

ValueError: could not convert string to float: 'maruti'

In [185]:
from sklearn import set_config
set_config(display='diagram')  
display(pipe)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('init',
                                                  Pipeline(steps=[('pre_preprocess',
                                                                   <__main__.DataframeFunctionTransformer object at 0x7fa658225970>)]),
                                                  ['name', 'fuel_type']),
                                                 ('numerical_prprocessing',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['km_driven']),
                                                 ('categoral_preprocessing',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['name', 'fuel_type'])])),
                ('model', LinearRegression())])

In [121]:
new_test = ['toyota','petrol',np.NaN]

In [122]:
new_ = pd.DataFrame(new_test).T

In [123]:
new_.columns = ['name','fuel_type', 'km_driven']

In [124]:
new_

,name,fuel_type,km_driven
0,toyota,petrol,NaN


In [125]:
pipe.predict(new_)

array([904492.0686354])

In [126]:
joblib.dump(pipe,'preprocess_model.pkl')

['preprocess_model.pkl']